# Foundations of Computer Science 2023 Project

We first import useful packages

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
from ipywidgets import IntProgress
from IPython.display import display
import time
import sys
import timeit

## 1. Extract all trips with busRoute 83

First of all we need to read the dataset

In [3]:
trips=pd.read_csv('metaData.csv')

Let's check what the dataset is like

In [4]:
trips

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500


Let us check what the different routes were

In [5]:
trips['busRoute']

0        -
1       31
2       33
3        -
4       72
        ..
1404    32
1405    83
1406    83
1407    N1
1408    N2
Name: busRoute, Length: 1409, dtype: object

Let's see if it recognizes 83 

In [6]:
trips['busRoute']==83

0       False
1       False
2       False
3       False
4       False
        ...  
1404    False
1405    False
1406    False
1407    False
1408    False
Name: busRoute, Length: 1409, dtype: bool

Since it doesn't (see lines 1405-1406) and there are routes named with letters, the data might be stored in strings

In [7]:
trips['busRoute']=='83'

0       False
1       False
2       False
3       False
4       False
        ...  
1404    False
1405     True
1406     True
1407    False
1408    False
Name: busRoute, Length: 1409, dtype: bool

Ok, that seems to be the case.

In [8]:
trips[trips['busRoute']=='83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


## 2. Extract all trips where busRoute is not a number

Let us define a function to check if the route is not a number. We have to do this because some routes are just missing (see lines 0 or 3 as example) and some others are letters and numbers (such as 1407-1408)

In [9]:
def check_not_number(route):
    m=re.match('^\d+\d$' ,route)
    if m:
        return False
    else: return True

Let's test it on some strings 

In [10]:
l='lol'
a='83'
b='N1'
c='189492749274398273'
d='18273t87217hh198718198'

In [11]:
check_not_number(l)

True

In [12]:
check_not_number(a)

False

In [13]:
#check_not_number(p)

In [14]:
check_not_number(b)

True

In [15]:
check_not_number(c)

False

In [16]:
check_not_number(d)

True

In [17]:
last_one='12N3'

In [18]:
check_not_number(last_one)

True

Since it seems to work we can use it on the actual dataset

In [19]:
trips[trips['busRoute'].apply(check_not_number)==True]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10T03:16:11Z,1557458171,2019-05-10T18:51:37Z,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13T03:10:23Z,1557717023,2019-05-13T23:16:13Z,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24T02:52:47Z,1558666367,2019-05-24T22:35:11Z,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


## 3. For each (busNumber, busRoute) pair, determine the number of trips

Let's check how many different buses number and routes we have:

In [20]:
trips['busNumber'].unique()

array([183, 208])

In [21]:
trips['busRoute'].unique()

array(['-', '31', '33', '72', '46', '32', '83', 'N4', 'N2', 'N1'],
      dtype=object)

Now we can just group by Number and Route and check how many lines we count, dropping the first one since it's referring to a missing busRoute.

In [22]:
trips.groupby(['busNumber', 'busRoute']).size().reset_index(name='Number of trips').drop(0)

,busNumber,busRoute,Number of trips
1,183,31,12
2,183,32,12
3,183,33,130
4,183,46,104
5,183,72,114
6,183,83,441
7,183,N1,10
8,183,N2,19
9,183,N4,11
10,208,31,5


This is in accord with point 2 of this project: the total number of trips for Route 83 is 441 for bus 183 and 405 for bus 208; that is a total of 846 trips, as the lines resulting in from question 1.

## 4. For each trip, compute the ratio between the energy consumption and the average number of passengers 

First of all let's check the data type of energyConsumption: if it is a string we'd have to convert it to a number

In [23]:
type(trips['energyConsumption'][0])

numpy.float64

No work needed here, we can just add a column with the requested information

In [24]:
trips['EnergyRatio']=trips['energyConsumption']/trips['itcs_numberOfPassengers_mean']

In [25]:
trips['EnergyRatio']

0       8.640500e+07
1       1.214747e+07
2       7.342794e+07
3       1.673328e+08
4       2.613190e+07
            ...     
1404    1.070215e+07
1405    5.338462e+07
1406    4.738047e+07
1407    2.247753e+07
1408    3.345769e+07
Name: EnergyRatio, Length: 1409, dtype: float64

## 5. For each station (itcs_stopName), determine the average number of passengers

We can first get a list of filenames from trips dataframe

In [26]:
namelist=trips['name']

let's try if everything is correct and it opens up the first one:

In [27]:
nome='ZTBus_compressed/'+namelist[0]+'.csv'
a=pd.read_csv(nome)
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30T03:18:58Z,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0
3,2019-04-30T03:18:59Z,1556594339,-1.087931,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,255000.0,0.0
4,2019-04-30T03:19:00Z,1556594340,-0.811985,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,253000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19520,2019-04-30T08:44:16Z,1556613856,-0.254938,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19521,2019-04-30T08:44:17Z,1556613857,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19522,2019-04-30T08:44:18Z,1556613858,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19523,2019-04-30T08:44:19Z,1556613859,-0.258701,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0


In [28]:
a['itcs_stopName'].unique()

array(['-', 'Zürich, Herdernstrasse', 'Zürich, Hardplatz',
       'Zürich, Güterbahnhof', 'Zürich, Bäckeranlage',
       'Zürich, Militär-/Langstrasse', 'Zürich, Kanonengasse',
       'Zürich, Sihlpost / HB', 'Zürich, Löwenplatz',
       'Zürich, Bahnhofplatz/HB', 'Zürich, Central', 'Zürich, Neumarkt',
       'Zürich, Kunsthaus', 'Zürich, Sprecherstrasse',
       'Zürich, Kreuzplatz', 'Zürich, Signaustrasse',
       'Zürich, Hegibachplatz', 'Zürich, Botanischer Garten',
       'Zürich, Höschgasse', 'Zürich, Fröhlichstrasse',
       'Zürich, Wildbachstrasse', 'Zürich, Bahnhof Tiefenbrunnen',
       'Zürich, Freiestrasse', 'Zürich, Klusplatz',
       'Zürich, Hölderlinsteig', 'Zürich, Klosbach', 'Zürich, Hofstrasse',
       'Zürich, Kirche Fluntern', 'Zürich, Hinterbergstrasse',
       'Zürich, Spyriplatz', 'Zürich, Bethanien', 'Zürich, Toblerplatz'],
      dtype=object)

We notice that we have many different stations name and quite some unnamed stops too; we surely don't need those though.

In [29]:
a=a.drop(a[a['itcs_stopName']=='-'].index)
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
246,2019-04-30T03:23:02Z,1556594582,46853.790,423.9126,1.303761,0.827037,0.148511,-,1.0,"Zürich, Herdernstrasse",...,0.917510,0.917510,0.917510,0,1,0,0,291.15,0.0,14819.530
322,2019-04-30T03:24:18Z,1556594658,54698.170,NaN,NaN,NaN,NaN,-,2.0,"Zürich, Hardplatz",...,1.242539,1.242539,1.242539,0,1,0,0,288.15,0.0,16939.120
406,2019-04-30T03:25:42Z,1556594742,33962.440,NaN,NaN,NaN,NaN,-,3.0,"Zürich, Güterbahnhof",...,0.610547,0.610547,0.610547,0,1,0,0,287.15,0.0,6230.346
508,2019-04-30T03:27:24Z,1556594844,59833.090,NaN,NaN,NaN,NaN,-,5.0,"Zürich, Bäckeranlage",...,1.052014,1.052014,1.052014,0,1,0,0,287.15,0.0,20058.230
564,2019-04-30T03:28:20Z,1556594900,11191.580,NaN,NaN,NaN,NaN,-,6.0,"Zürich, Militär-/Langstrasse",...,0.000000,0.000000,0.000000,0,1,0,0,286.15,0.0,12064.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18067,2019-04-30T08:20:03Z,1556612403,-9666.809,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Freiestrasse",...,8.818254,8.887699,8.858202,0,0,0,0,283.15,0.0,-4233.973
18099,2019-04-30T08:20:35Z,1556612435,-98946.970,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Klusplatz",...,4.914641,4.966725,4.937789,0,0,0,0,283.15,0.0,-22060.570
18161,2019-04-30T08:21:37Z,1556612497,21194.270,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Hölderlinsteig",...,0.000000,0.000000,0.000000,0,0,0,0,283.15,0.0,19751.250
18269,2019-04-30T08:23:25Z,1556612605,18611.560,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Klosbach",...,0.000000,0.000000,0.000000,0,0,0,0,283.15,0.0,17867.500


In [29]:
#nome=trips['name']
#for i in range(len(trips)):
#    nome[i]='ZTBUS_compressed/'+nome[i]+'.csv'
#    a=pd.read_csv(nome[i], usecols=['itcs_stopName'])
#    a=a.drop(a[a['itcs_stopName']=='-'].index)
#    stoplist=a['itcs_stopName'].unique()
    # stoplist=a[a['itcs_stopName']!='-']['itcs_stopName'].unique()
#    trips['StopList'][i]=stoplist
#    weights=[]
#    for stop in stoplist:
#        weights.append(str(a[a['itcs_stopName']==stop].count().item()))
#    trips['StopList_weights'][i] = weights

In [30]:
#trips['StopList'][0]==test[test['itcs_stopName']!='-']['itcs_stopName'].unique()

In [31]:
#def check_value (value, listL):
#    for i in range(len(listL)):
#        if value==listL[i]:
#            return i
#    return -1

In [32]:
#value=0
#listL=[1,2,3,4,5,6,0,1,2,4,6,8,6]
#print(check_value (value, listL))
#listL=[1,2,3,4,5,6,1,2,4,6,8,6]
#print(check_value (value, listL))
#listL=['a',2,'ciao',4,5,6,1,2,4,6,8,6]
#print(check_value (value, listL))
#listL=['a',2,'ciao',4,5,6,1,2,4,0,8,6]
#print(check_value (value, listL))
#value='ciao'
#print(check_value (value, listL))

In [33]:
#couples=[]
#stationlist=[]
#for j in range(trips['StopList'][0].size):
#    value=trips['StopList'][0][j]
#    k=int(trips['StopList_weights'][0][j])
#    a=trips['itcs_numberOfPassengers_mean'][0]*k
#    for i in range(len(trips)):
#        listL=trips['StopList'][i]
#        if check_value (value, listL)!=-1:
#            k+=int(trips['StopList_weights'][check_value (value, listL)][j])
#            a+=trips['itcs_numberOfPassengers_mean'][i]*int(trips['StopList_weights'][check_value (value, listL)][j])
#    couple=(value, a/k)
#    if a/k!=0:
#        couples.append(couple)
#    stationlist.append(value)

We can now modify namelist to grab the individual files from their Github directory. If you have the directory already downloaded, you can simply rename it to "ZTBus_compressed" and use instead the code from the next cell, to speed things up

In [34]:
#warnings.filterwarnings('ignore')
#for i in range(len(namelist)):
#    namelist[i]='https://github.com/CarloArpini/2023-FCS-Project/raw/main/ZTBus_compressed/'+namelist[i]+'.csv'

Insert your pathname in the cell below to indicate where you've stored the directory and its name. To get the path name you can, for example, move to said directory from terminal and interrogate it with < pwd >.

In [30]:
pathname=str('ZTBus_compressed/')

In [31]:
warnings.filterwarnings('ignore')
for i in range(len(namelist)):
    namelist[i]=pathname+namelist[i]+'.csv'

In [32]:
namelist

0       ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
1       ZTBus_compressed/B183_2019-04-30_13-22-07_2019...
2       ZTBus_compressed/B183_2019-05-01_05-58-51_2019...
3       ZTBus_compressed/B183_2019-05-03_02-50-21_2019...
4       ZTBus_compressed/B183_2019-05-03_15-41-57_2019...
                              ...                        
1404    ZTBus_compressed/B208_2022-12-06_14-43-49_2022...
1405    ZTBus_compressed/B208_2022-12-07_05-13-02_2022...
1406    ZTBus_compressed/B208_2022-12-08_05-22-20_2022...
1407    ZTBus_compressed/B208_2022-12-09_23-55-12_2022...
1408    ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
Name: name, Length: 1409, dtype: object

Now we introduce a snippet of code that will help us with question 6 - all will be explained there

In [33]:
numberbus=trips['busNumber']
k=0
while numberbus[k]==183:
    k+=1
count=0
for i in range(len(numberbus)):
    if numberbus[i]==183:
        count+=1
if k==count:
    print('the last row with busNumber value of 183 is row', k)
else:
    print(k, count, 'Something has gone wrong. Try and reload the notebook or check if you modified trips dataframe')
    k=0

the last row with busNumber value of 183 is row 864


Now we can concatenate each of the 1409 csv files together but dropping the unwanted lines (those with '-' as stopName), and making sure that we concatenate them all together and not subsequentially


In [37]:
start_time = timeit.default_timer()

previous=pd.read_csv(namelist[0])
previous.drop(previous[previous['itcs_stopName']=='-'].index, inplace=True)
previous['busNumber']=183
previous['namefile']=namelist[0]
frameslist=[previous]
max_count = len(trips)-1
f = IntProgress(min=0, max=max_count)
print('You can see progress here:')
display(f)
for i in range(len(trips)-1):
    a=pd.read_csv(namelist[i+1])
    a.drop(a[a['itcs_stopName']=='-'].index, inplace=True)
    # the following if-else condition will be useful for question 6 of the dataframe and will be explained 
    # there (as the snippet above)
    if i<k-1:
        a['busNumber']=183
    else:
        a['busNumber']=208
    a['namefile']=namelist[i+1]
    templist=[a]
    frameslist=frameslist+templist
    f.value += 1
previous=pd.concat(frameslist)
print('done!')

elapsed = timeit.default_timer() - start_time


You can see progress here:


IntProgress(value=0, max=1408)

done!
78.1360028340132


In [38]:
print(elapsed/60)

1.30226671390022


In [39]:
previous

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,busNumber,namefile
246,2019-04-30T03:23:02Z,1556594582,46853.790,423.9126,1.303761,0.827037,0.148511,-,1.0,"Zürich, Herdernstrasse",...,0.917510,0.0,1.0,0.0,0.0,291.1500,0.0,14819.53000,183,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
322,2019-04-30T03:24:18Z,1556594658,54698.170,NaN,NaN,NaN,NaN,-,2.0,"Zürich, Hardplatz",...,1.242539,0.0,1.0,0.0,0.0,288.1500,0.0,16939.12000,183,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
406,2019-04-30T03:25:42Z,1556594742,33962.440,NaN,NaN,NaN,NaN,-,3.0,"Zürich, Güterbahnhof",...,0.610547,0.0,1.0,0.0,0.0,287.1500,0.0,6230.34600,183,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
508,2019-04-30T03:27:24Z,1556594844,59833.090,NaN,NaN,NaN,NaN,-,5.0,"Zürich, Bäckeranlage",...,1.052014,0.0,1.0,0.0,0.0,287.1500,0.0,20058.23000,183,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
564,2019-04-30T03:28:20Z,1556594900,11191.580,NaN,NaN,NaN,NaN,-,6.0,"Zürich, Militär-/Langstrasse",...,0.000000,0.0,1.0,0.0,0.0,286.1500,0.0,12064.05000,183,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12542,2022-12-11T03:03:48Z,1670727828,12677.750,464.3928,0.000000,0.827626,0.148469,N2,9.0,"Zürich, Zehntenhausplatz",...,0.000000,0.0,0.0,1.0,0.0,274.1500,100040.5,10472.81000,208,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
12591,2022-12-11T03:04:37Z,1670727877,5376.588,458.3075,0.000000,0.827651,0.148482,N2,0.0,"Zürich, Bahnhof Affoltern",...,0.000000,0.0,0.0,1.0,0.0,274.1500,245000.0,489.56790,208,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
12636,2022-12-11T03:05:22Z,1670727922,-55071.570,456.9075,2.076901,0.827671,0.148520,N2,0.0,"Zürich, Fronwald",...,8.016577,0.0,0.0,0.0,0.0,274.1500,0.0,-9181.79800,208,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
12674,2022-12-11T03:06:00Z,1670727960,-22885.870,452.2150,0.357920,0.827700,0.148547,N2,0.0,"Zürich, Unteraffoltern",...,8.727656,0.0,0.0,0.0,0.0,274.1500,0.0,-5328.51800,208,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...


Now all that is left is to group by stop name and calculate the average

In [40]:
previous.groupby('itcs_stopName').mean('itcs_numberOfPassengers')['itcs_numberOfPassengers']

itcs_stopName
Zürich, Albisrank                 16.196598
Zürich, Albisriederplatz          25.453398
Zürich, Altes Krematorium         21.980348
Zürich, Bahnhof Affoltern          2.850746
Zürich, Bahnhof Altstetten         7.816580
                                    ...    
Zürich, Zentrum Witikon           17.255102
Zürich, Zweiackerstrasse           7.502203
Zürich, Zwielplatz                 2.049383
Zürich, Zwinglihaus               23.196668
Zürich,Kalkbreite/Bhf.Wiedikon    23.160748
Name: itcs_numberOfPassengers, Length: 149, dtype: float64

We can check that it is correct by calculating the number of unique stop names 

In [41]:
len(previous['itcs_stopName'].unique())

149

## 6. For each station, determine the buses that have stopped there at least once

We can now explain the snippet of code introduced in step 5. We noticed that the bus number was always either 183 or 208 and they seemed to be ordered: for this reason we can first check how many times 183 appears in the trips dataframe, and then check which is the first row with busNumber 208 (i.e. 864). If the two numbers are equals then the trips dataframe is ordered and contains the first N=864 rows referred to bus 183 and the latter 1409-N rows referred to bus 208; then we can go on and add a new busNumber column for each of the concatenated 1409 files where we simply insert either 183 or 208. At this point our database is provided with a busNumber associated to each row; then we can go on to group by stopName and bus number and check which buses stopped there at least once

In [42]:
previous[['itcs_stopName','busNumber']].groupby(['itcs_stopName', 'busNumber']).count().head(50)

Empty DataFrame
Columns: []
Index: [(Zürich, Albisrank, 183), (Zürich, Albisrank, 208), (Zürich, Albisriederplatz, 183), (Zürich, Albisriederplatz, 208), (Zürich, Altes Krematorium, 183), (Zürich, Altes Krematorium, 208), (Zürich, Bahnhof Affoltern, 183), (Zürich, Bahnhof Affoltern, 208), (Zürich, Bahnhof Altstetten, 183), (Zürich, Bahnhof Altstetten, 208), (Zürich, Bahnhof Hardbrücke, 183), (Zürich, Bahnhof Hardbrücke, 208), (Zürich, Bahnhof Tiefenbrunnen, 183), (Zürich, Bahnhof Tiefenbrunnen, 208), (Zürich, Bahnhof Wipkingen, 183), (Zürich, Bahnhof Wipkingen, 208), (Zürich, Bahnhofplatz/HB, 183), (Zürich, Bahnhofplatz/HB, 208), (Zürich, Bahnhofquai/HB, 183), (Zürich, Bahnhofquai/HB, 208), (Zürich, Baslerstrasse, 183), (Zürich, Baslerstrasse, 208), (Zürich, Berghaldenstrasse, 183), (Zürich, Berghaldenstrasse, 208), (Zürich, Bertastrasse, 183), (Zürich, Bertastrasse, 208), (Zürich, Besenrainstrasse, 183), (Zürich, Besenrainstrasse, 208), (Zürich, Bethanien, 183), (Zürich, Bethanien, 208), (Zürich, Birchdörfli, 183), (Zürich, Birchdörfli, 208), (Zürich, Botanischer Garten, 183), (Zürich, Botanischer Garten, 208), (Zürich, Bristenstrasse, 183), (Zürich, Bristenstrasse, 208), (Zürich, Brunau/Mutschellenstr., 183), (Zürich, Brunau/Mutschellenstr., 208), (Zürich, Bucheggplatz, 183), (Zürich, Bucheggplatz, 208), (Zürich, Bäckeranlage, 183), (Zürich, Bäckeranlage, 208), (Zürich, Carl-Spitteler-Strasse, 183), (Zürich, Central, 183), (Zürich, Central, 208), (Zürich, Drusbergstrasse, 183), (Zürich, Drusbergstrasse, 208), (Zürich, Dunkelhölzli, 183), (Zürich, Dunkelhölzli, 208), (Zürich, Einfangstrasse, 183)]

As a simple check, let's query our database looking for which bus stopped at Zürich, Carl-Spitteler-Strasse which seems to be one of the few stops only served by bus 183

In [43]:
previous[previous['itcs_stopName']=='Zürich, Carl-Spitteler-Strasse']['busNumber'].unique()

array([183])

## 7. For each station, determine the buses that have stopped there at least ten times

This is fairly easy, we just have to group by stop names and bus number as before and then we can count the number of stops

In [44]:
stopsnumber=previous.groupby(['itcs_stopName', 'busNumber']).size().reset_index(name='number of stops')
stopsnumber[stopsnumber['number of stops']>=10]

,itcs_stopName,busNumber,number of stops
0,"Zürich, Albisrank",183,6324
1,"Zürich, Albisrank",208,6199
2,"Zürich, Albisriederplatz",183,11218
3,"Zürich, Albisriederplatz",208,7633
4,"Zürich, Altes Krematorium",183,10022
...,...,...,...
291,"Zürich, Zwielplatz",183,77
293,"Zürich, Zwinglihaus",183,2791
294,"Zürich, Zwinglihaus",208,1231
295,"Zürich,Kalkbreite/Bhf.Wiedikon",183,279


## 8. Click on the following link 

https://www.youtube.com/watch?v=dQw4w9WgXcQ

## 9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance

We can just group by route and bus number and then sum over energy consumption and driven distance:

In [45]:
grouped = trips.groupby(['busRoute','busNumber']).sum()[['energyConsumption', 'drivenDistance']]
grouped

energyConsumption  drivenDistance
busRoute busNumber                                   
-        183             1.245498e+10      1999119.75
31       183             1.387782e+10      2279952.00
         208             5.535865e+09      1046306.68
32       183             1.393320e+10      2256540.10
         208             1.283340e+10      2337064.21
33       183             1.761996e+11     29510481.41
         208             3.474842e+10      6161892.39
46       183             1.026237e+11     18224657.94
         208             2.337366e+10      4186493.70
72       183             1.394368e+11     23637238.45
         208             5.100382e+10      9427138.15
83       183             2.995540e+11     51249813.70
         208             2.907837e+11     49965928.79
N1       183             4.217283e+09       705122.71
         208             2.316328e+09       410691.93
N2       183             7.929098e+09      1390618.35
         208             7.913191e+09      1464008.67
N4       183             4.720068e+09       766868.89
         208             3.152512e+09       519599.97

We notice that some routes are missing so let's take care of that

In [46]:
trips_new=trips.drop(trips[trips['busRoute']=='-'].index)
grouped = trips_new.groupby(['busRoute','busNumber']).sum()[['energyConsumption', 'drivenDistance']]
grouped

energyConsumption  drivenDistance
busRoute busNumber                                   
31       183             1.387782e+10      2279952.00
         208             5.535865e+09      1046306.68
32       183             1.393320e+10      2256540.10
         208             1.283340e+10      2337064.21
33       183             1.761996e+11     29510481.41
         208             3.474842e+10      6161892.39
46       183             1.026237e+11     18224657.94
         208             2.337366e+10      4186493.70
72       183             1.394368e+11     23637238.45
         208             5.100382e+10      9427138.15
83       183             2.995540e+11     51249813.70
         208             2.907837e+11     49965928.79
N1       183             4.217283e+09       705122.71
         208             2.316328e+09       410691.93
N2       183             7.929098e+09      1390618.35
         208             7.913191e+09      1464008.67
N4       183             4.720068e+09       766868.89
         208             3.152512e+09       519599.97

Now let's just create a new column with the wanted ratio:

In [47]:
grouped['ratio'] = grouped['energyConsumption'] / grouped['drivenDistance']
grouped

energyConsumption  drivenDistance        ratio
busRoute busNumber                                                
31       183             1.387782e+10      2279952.00  6086.891785
         208             5.535865e+09      1046306.68  5290.862713
32       183             1.393320e+10      2256540.10  6174.585331
         208             1.283340e+10      2337064.21  5491.247671
33       183             1.761996e+11     29510481.41  5970.747507
         208             3.474842e+10      6161892.39  5639.244537
46       183             1.026237e+11     18224657.94  5631.035959
         208             2.337366e+10      4186493.70  5583.110707
72       183             1.394368e+11     23637238.45  5899.029643
         208             5.100382e+10      9427138.15  5410.318878
83       183             2.995540e+11     51249813.70  5844.977154
         208             2.907837e+11     49965928.79  5819.638891
N1       183             4.217283e+09       705122.71  5980.920966
         208             2.316328e+09       410691.93  5640.061883
N2       183             7.929098e+09      1390618.35  5701.850332
         208             7.913191e+09      1464008.67  5405.152963
N4       183             4.720068e+09       766868.89  6154.986937
         208             3.152512e+09       519599.97  6067.190112

## 10. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe

First we extract maximum value of our ratio for each route:

In [48]:
grouped_max= grouped.loc[grouped['ratio'].groupby('busRoute').idxmax()]
grouped_max

,,energyConsumption,drivenDistance,ratio
busRoute,busNumber,,,
31,183,1.387782e+10,2279952.00,6086.891785
32,183,1.393320e+10,2256540.10,6174.585331
33,183,1.761996e+11,29510481.41,5970.747507
46,183,1.026237e+11,18224657.94,5631.035959
72,183,1.394368e+11,23637238.45,5899.029643
83,183,2.995540e+11,51249813.70,5844.977154
N1,183,4.217283e+09,705122.71,5980.920966
N2,183,7.929098e+09,1390618.35,5701.850332
N4,183,4.720068e+09,766868.89,6154.986937


Then we do the same for the minimum ratio value over the routes:

In [49]:
grouped_min=grouped.loc[grouped['ratio'].groupby('busRoute').idxmin()]
grouped_min

,,energyConsumption,drivenDistance,ratio
busRoute,busNumber,,,
31,208,5.535865e+09,1046306.68,5290.862713
32,208,1.283340e+10,2337064.21,5491.247671
33,208,3.474842e+10,6161892.39,5639.244537
46,208,2.337366e+10,4186493.70,5583.110707
72,208,5.100382e+10,9427138.15,5410.318878
83,208,2.907837e+11,49965928.79,5819.638891
N1,208,2.316328e+09,410691.93,5640.061883
N2,208,7.913191e+09,1464008.67,5405.152963
N4,208,3.152512e+09,519599.97,6067.190112


We then merge the two:

In [50]:
grouped_tot = pd.merge(grouped_max, grouped_min, on='busRoute')
grouped_tot

,energyConsumption_x,drivenDistance_x,ratio_x,energyConsumption_y,drivenDistance_y,ratio_y
busRoute,,,,,,
31,1.387782e+10,2279952.00,6086.891785,5.535865e+09,1046306.68,5290.862713
32,1.393320e+10,2256540.10,6174.585331,1.283340e+10,2337064.21,5491.247671
33,1.761996e+11,29510481.41,5970.747507,3.474842e+10,6161892.39,5639.244537
46,1.026237e+11,18224657.94,5631.035959,2.337366e+10,4186493.70,5583.110707
72,1.394368e+11,23637238.45,5899.029643,5.100382e+10,9427138.15,5410.318878
83,2.995540e+11,51249813.70,5844.977154,2.907837e+11,49965928.79,5819.638891
N1,4.217283e+09,705122.71,5980.920966,2.316328e+09,410691.93,5640.061883
N2,7.929098e+09,1390618.35,5701.850332,7.913191e+09,1464008.67,5405.152963
N4,4.720068e+09,766868.89,6154.986937,3.152512e+09,519599.97,6067.190112


And now we have the difference simply by:

In [51]:
grouped_tot['ratio_x'] - grouped_tot['ratio_y']

busRoute
31    796.029072
32    683.337660
33    331.502969
46     47.925252
72    488.710765
83     25.338263
N1    340.859083
N2    296.697369
N4     87.796825
dtype: float64

## 11. Find the bus maximizing the difference computed in the previous point

## 12. Extract the rows of the details such that the gnss_altitude differs from the value in the preceding row. Store also the difference in the variable altitude_variation

Let's check what gnss_altitude is like in a single dataframe, say the first of the 1409 files

In [52]:
a=pd.read_csv(nome)
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30T03:18:58Z,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0
3,2019-04-30T03:18:59Z,1556594339,-1.087931,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,255000.0,0.0
4,2019-04-30T03:19:00Z,1556594340,-0.811985,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,253000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19520,2019-04-30T08:44:16Z,1556613856,-0.254938,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19521,2019-04-30T08:44:17Z,1556613857,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19522,2019-04-30T08:44:18Z,1556613858,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19523,2019-04-30T08:44:19Z,1556613859,-0.258701,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0


Since there are a lot of missing NaN rows, let's see how many are left when we remove those and decide if it's worth removing them

In [53]:
a[a['gnss_altitude'].isnull()!=True]

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
142,2019-04-30T03:21:18Z,1556594478,16140.750,397.8000,1.432915,0.827004,0.148489,-,NaN,-,...,3.218779,2.652735,3.118049,0,1,0,0,292.15,0.0,3437.261000
143,2019-04-30T03:21:19Z,1556594479,17476.710,397.8000,1.432915,0.827002,0.148489,-,NaN,-,...,3.180786,2.788640,3.146064,0,1,0,0,292.15,0.0,3621.687000
144,2019-04-30T03:21:20Z,1556594480,17800.860,397.8000,1.432915,0.827003,0.148489,-,NaN,-,...,3.214131,3.008278,3.233972,0,1,0,0,292.15,0.0,3644.703000
145,2019-04-30T03:21:21Z,1556594481,4461.355,397.8000,1.432915,0.827003,0.148489,-,NaN,-,...,3.133200,3.015251,3.167922,0,1,0,0,292.15,0.0,-2.268991
146,2019-04-30T03:21:22Z,1556594482,28584.730,397.8000,1.432915,0.827002,0.148487,-,NaN,-,...,3.184125,3.114681,3.236208,0,1,0,0,292.15,0.0,6621.516000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2019-04-30T03:25:12Z,1556594712,28578.890,399.5968,2.136395,0.826951,0.148684,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,119629.6,-6.655452
377,2019-04-30T03:25:13Z,1556594713,27972.820,399.3949,2.143442,0.826950,0.148686,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,245646.2,-2.601404
378,2019-04-30T03:25:14Z,1556594714,28028.800,399.0949,2.153736,0.826949,0.148688,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,245651.0,-2.601404
379,2019-04-30T03:25:15Z,1556594715,28376.830,398.7966,2.153825,0.826948,0.148690,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,244819.6,0.000000


It definetly is, so we can grab the index and remove those rows as follows:

In [54]:
a.drop(a[a['gnss_altitude'].isnull()==True].index)


,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
142,2019-04-30T03:21:18Z,1556594478,16140.750,397.8000,1.432915,0.827004,0.148489,-,NaN,-,...,3.218779,2.652735,3.118049,0,1,0,0,292.15,0.0,3437.261000
143,2019-04-30T03:21:19Z,1556594479,17476.710,397.8000,1.432915,0.827002,0.148489,-,NaN,-,...,3.180786,2.788640,3.146064,0,1,0,0,292.15,0.0,3621.687000
144,2019-04-30T03:21:20Z,1556594480,17800.860,397.8000,1.432915,0.827003,0.148489,-,NaN,-,...,3.214131,3.008278,3.233972,0,1,0,0,292.15,0.0,3644.703000
145,2019-04-30T03:21:21Z,1556594481,4461.355,397.8000,1.432915,0.827003,0.148489,-,NaN,-,...,3.133200,3.015251,3.167922,0,1,0,0,292.15,0.0,-2.268991
146,2019-04-30T03:21:22Z,1556594482,28584.730,397.8000,1.432915,0.827002,0.148487,-,NaN,-,...,3.184125,3.114681,3.236208,0,1,0,0,292.15,0.0,6621.516000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2019-04-30T03:25:12Z,1556594712,28578.890,399.5968,2.136395,0.826951,0.148684,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,119629.6,-6.655452
377,2019-04-30T03:25:13Z,1556594713,27972.820,399.3949,2.143442,0.826950,0.148686,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,245646.2,-2.601404
378,2019-04-30T03:25:14Z,1556594714,28028.800,399.0949,2.153736,0.826949,0.148688,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,245651.0,-2.601404
379,2019-04-30T03:25:15Z,1556594715,28376.830,398.7966,2.153825,0.826948,0.148690,-,NaN,-,...,0.000000,0.000000,0.000000,0,1,0,0,287.15,244819.6,0.000000


Now let's not forget we also have to drop rows where the value of gnss_altitude is equal to the one above; we can use the diff() method applied to the dataframe to get the index of those rows and remove them. 
Then we can implement the following idea: we open all files as we did for question 5 and for each file we drop NaN rows and rows where the diff is zero. Then we concat everything in a single dataframe, as was done in step 5; here though using concat for all dataframes is of vital importance though because if concatenated subsequentially (that is concatenating the first two together, then the one resulting with the third and so on) would result in calling up a dataframe with tens of millions of lines hunderds of times for the last of the 1409 files

In [70]:
start_time = timeit.default_timer()

height_nonan=pd.read_csv(namelist[0])
height_nonan.drop(height_nonan[height_nonan['gnss_altitude'].isnull()==True].index, inplace=True)
height_nonan.drop(height_nonan[height_nonan['gnss_altitude'].diff()==0].index, inplace=True)
height_nonan['namefile']=namelist[0]
frameslist=[height_nonan]
max_count = len(trips)-1
f = IntProgress(min=0, max=max_count)
print('You can see progress here:')
display(f)
for i in range(len(trips)-1):
    a=pd.read_csv(namelist[i+1])
    a.drop(a[a['gnss_altitude'].isnull()==True].index, inplace=True)
    a.drop(a[a['gnss_altitude'].diff()==0].index, inplace=True)
    # we now add a line of code useful to complete the next question, so it will be explained there
    a['namefile']=namelist[i+1]
    templist=[a]
    frameslist=frameslist+templist
    f.value += 1
height_nonan=pd.concat(frameslist)
# now reset index to then be able to work with diff once again to create altitude_variation variable
height_nonan.reset_index(inplace=True)
print('Done!')

elapsed = timeit.default_timer() - start_time

You can see progress here:


IntProgress(value=0, max=1408)

Done!


In [71]:
print(elapsed/60)

1.586167863199565


In [72]:
height_nonan

,index,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,...,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,namefile
0,142,2019-04-30T03:21:18Z,1556594478,16140.750000,397.8000,1.432915,0.827004,0.148489,-,NaN,...,2.652735,3.118049,0.0,1.0,0.0,0.0,292.15,0.0,3437.261,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
1,148,2019-04-30T03:21:24Z,1556594484,32607.160000,397.8299,1.474908,0.827001,0.148487,-,NaN,...,3.663105,3.663105,0.0,1.0,0.0,0.0,292.15,0.0,7033.599,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
2,149,2019-04-30T03:21:25Z,1556594485,39229.610000,400.3140,4.932499,0.827002,0.148485,-,NaN,...,3.898375,3.901954,0.0,1.0,0.0,0.0,292.15,0.0,7572.098,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
3,150,2019-04-30T03:21:26Z,1556594486,54537.210000,401.6892,4.258572,0.827000,0.148484,-,NaN,...,4.239702,4.257063,0.0,1.0,0.0,0.0,292.15,0.0,10395.160,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
4,151,2019-04-30T03:21:27Z,1556594487,52785.540000,400.4860,4.065155,0.826998,0.148484,-,NaN,...,4.712781,4.698999,0.0,1.0,0.0,0.0,292.15,0.0,9944.972,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40770669,14031,2022-12-11T03:28:37Z,1670729317,3214.522000,416.0540,0.365378,0.826997,0.148459,-,NaN,...,0.000000,0.000000,1.0,0.0,1.0,1.0,278.15,245833.3,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
40770670,14042,2022-12-11T03:28:48Z,1670729328,18.181250,413.7306,0.288575,0.827004,0.148459,-,NaN,...,0.000000,0.000000,1.0,0.0,0.0,1.0,278.15,0.0,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
40770671,14054,2022-12-11T03:29:00Z,1670729340,-0.051249,410.1028,1.918733,0.826986,0.148482,-,NaN,...,0.000000,0.000000,1.0,0.0,0.0,1.0,278.15,0.0,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...
40770672,14055,2022-12-11T03:29:01Z,1670729341,0.216363,410.1916,2.195930,0.826985,0.148488,-,NaN,...,0.000000,0.000000,1.0,0.0,0.0,1.0,278.15,0.0,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...


Let's now create the new altitude_variation variable; we expect clearly the first row to be some gibberish since there is no row above that and for this reason diff would be computing the difference of a value and something outside of our control (so we either expect a nonsensical value if diff accesses whatever is stored somewhere or maybe a NaN if there are checks to ensure this does not happen; either way it is to be discarded)

In [73]:
height_nonan['altitude_variation']=height_nonan['gnss_altitude'].diff()
height_nonan

,index,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,...,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,namefile,altitude_variation
0,142,2019-04-30T03:21:18Z,1556594478,16140.750000,397.8000,1.432915,0.827004,0.148489,-,NaN,...,3.118049,0.0,1.0,0.0,0.0,292.15,0.0,3437.261,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,NaN
1,148,2019-04-30T03:21:24Z,1556594484,32607.160000,397.8299,1.474908,0.827001,0.148487,-,NaN,...,3.663105,0.0,1.0,0.0,0.0,292.15,0.0,7033.599,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,0.0299
2,149,2019-04-30T03:21:25Z,1556594485,39229.610000,400.3140,4.932499,0.827002,0.148485,-,NaN,...,3.901954,0.0,1.0,0.0,0.0,292.15,0.0,7572.098,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,2.4841
3,150,2019-04-30T03:21:26Z,1556594486,54537.210000,401.6892,4.258572,0.827000,0.148484,-,NaN,...,4.257063,0.0,1.0,0.0,0.0,292.15,0.0,10395.160,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,1.3752
4,151,2019-04-30T03:21:27Z,1556594487,52785.540000,400.4860,4.065155,0.826998,0.148484,-,NaN,...,4.698999,0.0,1.0,0.0,0.0,292.15,0.0,9944.972,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,-1.2032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40770669,14031,2022-12-11T03:28:37Z,1670729317,3214.522000,416.0540,0.365378,0.826997,0.148459,-,NaN,...,0.000000,1.0,0.0,1.0,1.0,278.15,245833.3,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,-1.1372
40770670,14042,2022-12-11T03:28:48Z,1670729328,18.181250,413.7306,0.288575,0.827004,0.148459,-,NaN,...,0.000000,1.0,0.0,0.0,1.0,278.15,0.0,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,-2.3234
40770671,14054,2022-12-11T03:29:00Z,1670729340,-0.051249,410.1028,1.918733,0.826986,0.148482,-,NaN,...,0.000000,1.0,0.0,0.0,1.0,278.15,0.0,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,-3.6278
40770672,14055,2022-12-11T03:29:01Z,1670729341,0.216363,410.1916,2.195930,0.826985,0.148488,-,NaN,...,0.000000,1.0,0.0,0.0,1.0,278.15,0.0,0.000,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,0.0888


## 13. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of altitude_variation

Since we added a new namefile column, this is fairly easy: we can just group by namefile and proceed to do the sum. The tricky part is to first of all drop the unwanted nonsensical first line and then transform all wanted values to their absolute value counterpart

In [74]:
height_nonan.drop(0, inplace=True)

It's best to store the grouped database to then apply abs to it and eventualy regroup it by namefile to sum over it

In [75]:
detail_grouped=height_nonan.groupby('namefile')

In [76]:
detail_grouped['altitude_variation'].apply(abs)

namefile                                                                   
ZTBus_compressed/B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv  1           0.0299
                                                                   2           2.4841
                                                                   3           1.3752
                                                                   4           1.2032
                                                                   5           1.3668
                                                                                ...  
ZTBus_compressed/B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv  40770669    1.1372
                                                                   40770670    2.3234
                                                                   40770671    3.6278
                                                                   40770672    0.0888
                                                                

In [77]:
detail_grouped['altitude_variation'].apply(abs).groupby('namefile').sum()

namefile
ZTBus_compressed/B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv      188.3500
ZTBus_compressed/B183_2019-04-30_13-22-07_2019-04-30_17-54-02.csv     4299.3578
ZTBus_compressed/B183_2019-05-01_05-58-51_2019-05-01_22-32-30.csv      804.6894
ZTBus_compressed/B183_2019-05-03_02-50-21_2019-05-03_05-53-20.csv     4271.1584
ZTBus_compressed/B183_2019-05-03_15-41-57_2019-05-03_23-06-24.csv    13543.2470
                                                                        ...    
ZTBus_compressed/B208_2022-12-06_14-43-49_2022-12-06_18-22-52.csv     2330.1891
ZTBus_compressed/B208_2022-12-07_05-13-02_2022-12-07_19-19-53.csv     8654.8137
ZTBus_compressed/B208_2022-12-08_05-22-20_2022-12-08_18-39-15.csv     8136.7387
ZTBus_compressed/B208_2022-12-09_23-55-12_2022-12-10_03-24-28.csv     2810.9444
ZTBus_compressed/B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv     3609.6485
Name: altitude_variation, Length: 1409, dtype: float64

let's now check that everything is correct with a different manual method

In [78]:
height_nonan[height_nonan['namefile']==namelist[0]]['altitude_variation'].abs().sum()

188.3500000000006

## 14. For each month of the year, compute the average ambient temperature

First of all we transform the temperature to celsius, for easier understanding:

In [79]:
warnings.filterwarnings('ignore')
for i in range(len(trips)):
    trips['temperature_ambient_mean'][i]=trips['temperature_ambient_mean'][i]-273.15

Now let's set the date:

In [80]:
trips['startTime_iso']=pd.to_datetime(trips['startTime_iso'])

Let's put the month in a new columm:

In [81]:
trips['month']=trips['startTime_iso'].dt.month

And we have the result:

In [82]:
trips.groupby('month')['temperature_ambient_mean'].mean().reset_index(name='avg_temp')

,month,avg_temp
0,1,5.808474
1,2,7.379928
2,3,10.169157
3,4,13.458216
4,5,16.901582
5,6,22.993297
6,7,24.226967
7,8,22.697943
8,9,19.212038
9,10,14.564072


## 15. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times

In [83]:
a=pd.read_csv(namelist[0])
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30T03:18:58Z,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0
3,2019-04-30T03:18:59Z,1556594339,-1.087931,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,255000.0,0.0
4,2019-04-30T03:19:00Z,1556594340,-0.811985,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,253000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19520,2019-04-30T08:44:16Z,1556613856,-0.254938,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19521,2019-04-30T08:44:17Z,1556613857,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19522,2019-04-30T08:44:18Z,1556613858,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19523,2019-04-30T08:44:19Z,1556613859,-0.258701,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0


We see that there are two columns of zeros and ones which are associated with what we can compute. We have to first transform the datetime to the correct format though

In [84]:
a['time_iso']=pd.to_datetime(a['time_iso'])
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30 03:18:56+00:00,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30 03:18:57+00:00,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30 03:18:58+00:00,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0
3,2019-04-30 03:18:59+00:00,1556594339,-1.087931,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,255000.0,0.0
4,2019-04-30 03:19:00+00:00,1556594340,-0.811985,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,253000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19520,2019-04-30 08:44:16+00:00,1556613856,-0.254938,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19521,2019-04-30 08:44:17+00:00,1556613857,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19522,2019-04-30 08:44:18+00:00,1556613858,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19523,2019-04-30 08:44:19+00:00,1556613859,-0.258701,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0


Now we notice something: for each row, the time advances (or so it seems) of exactly one second. If this is true, the total number of seconds a specific brake is active is just the total number of rows with value 1! Let's check if this is the case

In [85]:
a['seconds']=a['time_iso'].dt.second

In [86]:
a['time_iso'][19524]-a['time_iso'][0]

Timedelta('0 days 05:25:24')

In [87]:
(a['time_iso'][19524]-a['time_iso'][0]).total_seconds()

19524.0

Ok so it seems to be as expected, but we have to be careful since the number of seconds is one less then the lenght of the dataframe; this is because we are querying for the number of seconds in the difference of two time values; as such, the difference returns the number of seconds minus one. Think of it like this: if I have to count the number of 5 balls enumerated 1 trough 5, that is 5 balls, I could ask for the difference between the biggest number on one ball and the lowest; but that would result in 4. So I have to take in account one more, or in other words the difference does not take in account the number of numbers between two values but only the values of such numbers

Now the concept is simple: we read all of our usual 1409 files but stopping at k, which was the last value associated to bus number 183. For each of those files we create two dataframes, respectively fropping rows wich contain zero values for either status_parkBrakeIsActive or status_haltBrakeIsActive. This way we can then just count how many rows we have and get the number of seconds for bus 183 for both parked and halted times

In [88]:
start_time = timeit.default_timer()

lenght_of_trips=[]
bus183_park=pd.read_csv(namelist[0])
bus183_halt=bus183_park
bus183_park.drop(bus183_park[bus183_park['status_parkBrakeIsActive']==0].index, inplace=True)
bus183_halt.drop(bus183_halt[bus183_halt['status_haltBrakeIsActive']==0].index, inplace=True)
frameslist=[bus183_park]
frameslist2=[bus183_halt]
# we insert a lenght_of_trips list to do some cross checks afterwards
lenght_of_trips.append(len(bus183_park))
max_count = k-1
f = IntProgress(min=0, max=max_count)
print('You can see progress here:')
display(f)
for i in range(k-1):
    a=pd.read_csv(namelist[i+1])
    b=a
    lenght_of_trips.append(len(a))
    a.drop(a[a['status_parkBrakeIsActive']==0].index, inplace=True)
    templist=[a]
    frameslist=frameslist+templist
    b.drop(b[b['status_haltBrakeIsActive']==0].index, inplace=True)
    templist2=[b]
    frameslist2=frameslist2+templist2
    f.value += 1
bus183_park=pd.concat(frameslist)
bus183_park.reset_index(inplace=True)
bus183_halt=pd.concat(frameslist2)
bus183_halt.reset_index(inplace=True)
print('Done!')


elapsed = timeit.default_timer() - start_time
print(elapsed)

You can see progress here:


IntProgress(value=0, max=863)

Done!
53.108316708006896


Let's now do the same for bus 208

In [89]:
%timeit
bus208_park=pd.read_csv(namelist[0])
bus208_halt=bus208_park
bus208_park.drop(bus208_park[bus208_park['status_parkBrakeIsActive']==0].index, inplace=True)
bus208_halt.drop(bus208_halt[bus208_halt['status_haltBrakeIsActive']==0].index, inplace=True)
frameslist=[bus208_park]
frameslist2=[bus208_halt]
max_count = len(trips)-1
f = IntProgress(min=k, max=max_count)
print('You can see progress here:')
display(f)
i=k
while i<max_count:
    a=pd.read_csv(namelist[i+1])
    b=a
    a.drop(a[a['status_parkBrakeIsActive']==0].index, inplace=True)
    templist=[a]
    frameslist=frameslist+templist
    b.drop(b[b['status_haltBrakeIsActive']==0].index, inplace=True)
    templist2=[b]
    frameslist2=frameslist2+templist2
    f.value += 1
    i+=1
bus208_park=pd.concat(frameslist)
bus208_park.reset_index(inplace=True)
bus208_halt=pd.concat(frameslist2)
bus208_halt.reset_index(inplace=True)
print('Done!')

You can see progress here:


IntProgress(value=864, max=1408, min=864)

Done!


In [90]:
union={'bus Number':[183, 208],
       'total time halt brake active': [len(bus183_halt['status_haltBrakeIsActive']),len(bus208_halt['status_haltBrakeIsActive'])],
       'total time park brake active': [len(bus183_park['status_parkBrakeIsActive']), len(bus208_park['status_parkBrakeIsActive'])]}
union_df=pd.DataFrame(data=union)
union_df['ratio']=union_df['total time halt brake active']/union_df['total time park brake active']
union_df

,bus Number,total time halt brake active,total time park brake active,ratio
0,183,2426302,2426302,1.0
1,208,1816878,1816878,1.0


Just to get a sense of scale since those numbers do look huge, let's check the maximum, median and average lenght of trips; remember that their lenght is essentially the time in seconds it took to complete the trip, for bus 183:

In [91]:
print('the maximum lenght was ', max(lenght_of_trips)/60,
      'minutes,\nthe median lenght was ', np.median(lenght_of_trips)/60,
      'minutes,\nwhile the average was ', sum(lenght_of_trips)/(len(lenght_of_trips)*60), ' minutes')

the maximum lenght was  1436.4666666666667 minutes,
the median lenght was  553.2 minutes,
while the average was  595.4652584876543  minutes


In [92]:
print('the average halt brake total duration for a trip was instead',
      (len(bus183_halt['status_haltBrakeIsActive']))/(k*60), ' minutes' )

the average halt brake total duration for a trip was instead 46.80366512345679  minutes


So as such it does make sense that over trips of about 10 hours total halting brake time was about 2 hours.

## 16. For each pair of stops that are consecutive in some trip, compute the average speed achieved when going from the first to the second stop

posso usare -1?

In [117]:
a=pd.read_csv(namelist[0], usecols=['itcs_stopName', 'odometry_vehicleSpeed'])

In [118]:
min1=-3e108

In [119]:
a

,itcs_stopName,odometry_vehicleSpeed
0,-,0.0
1,-,0.0
2,-,0.0
3,-,0.0
4,-,0.0
...,...,...
19520,-,0.0
19521,-,0.0
19522,-,0.0
19523,-,0.0


In [120]:
len(a)

19525

In [122]:

a['stop pair']=0
a['average_speed']=min1
i=0
templist=[]
speedtemp=np.array([])
stopsnumber=0
while i<len(a):
    if a['itcs_stopName'][i]!='-':
        templist.append(a['itcs_stopName'][i])
        stopsnumber+=1
        if stopsnumber==a[a['itcs_stopName']!='-']['itcs_stopName'].count():
            i=len(a)-1
    if templist!=[]:
        speedtemp=np.append(speedtemp, a['odometry_vehicleSpeed'][i])
    if(len(templist)==2):
        a['stop pair'][i]=templist
        a['average_speed'][i]=np.mean(speedtemp)
        templist=[]
        templist.append(a['itcs_stopName'][i])
        speedtemp=np.array([])
        speedtemp=np.append(speedtemp, a['odometry_vehicleSpeed'][i])
    i+=1
a.drop(a[a['average_speed']==min1].index, inplace=True)
a.drop(['itcs_stopName', 'odometry_vehicleSpeed'], axis=1, inplace=True)

In [123]:
a

,stop pair,average_speed
322,"[Zürich, Herdernstrasse, Zürich, Hardplatz]",6.640177
406,"[Zürich, Hardplatz, Zürich, Güterbahnhof]",5.376908
508,"[Zürich, Güterbahnhof, Zürich, Bäckeranlage]",4.130239
564,"[Zürich, Bäckeranlage, Zürich, Militär-/Langst...",6.915111
619,"[Zürich, Militär-/Langstrasse, Zürich, Kanonen...",4.642477
...,...,...
18067,"[Zürich, Hegibachplatz, Zürich, Freiestrasse]",6.157266
18099,"[Zürich, Freiestrasse, Zürich, Klusplatz]",9.150786
18161,"[Zürich, Klusplatz, Zürich, Hölderlinsteig]",3.932883
18269,"[Zürich, Hölderlinsteig, Zürich, Klosbach]",4.749490


In [126]:
times=[]
stops_and_average=pd.read_csv(namelist[0], usecols=['itcs_stopName', 'odometry_vehicleSpeed'])
stops_and_average['namefile']=namelist[0]
stops_and_average['stop_pair']=0
stops_and_average['average_speed']=min1
i=0
speedtemp=np.array([])
nametemp=[]
stopsnumber=0
while i<len(stops_and_average):
    if stops_and_average['itcs_stopName'][i]!='-':
        nametemp.append(stops_and_average['itcs_stopName'][i])
        stopsnumber+=1
        if stopsnumber==stops_and_average[stops_and_average['itcs_stopName']!='-']['itcs_stopName'].count():
            i=len(stops_and_average)-1
    if nametemp!=[]:
        speedtemp=np.append(speedtemp, stops_and_average['odometry_vehicleSpeed'][i])
    if(len(nametemp)==2):
        stops_and_average['stop_pair'][i]=nametemp
        stops_and_average['average_speed'][i]=np.mean(speedtemp)
        nametemp=[]
        nametemp.append(stops_and_average['itcs_stopName'][i])
        speedtemp=np.array([])
        speedtemp=np.append(speedtemp, stops_and_average['odometry_vehicleSpeed'][i])
    i+=1
stops_and_average.drop(stops_and_average[stops_and_average['average_speed']==min1].index, inplace=True)
stops_and_average.drop(['itcs_stopName', 'odometry_vehicleSpeed'], axis=1, inplace=True)

frameslist=[stops_and_average]
max_count = len(trips)-1
f = IntProgress(min=0, max=max_count)
print('You can see progress here:')
display(f)
for i in range(max_count):
    a=pd.read_csv(namelist[i+1], usecols=['itcs_stopName', 'odometry_vehicleSpeed'])
    a['namefile']=namelist[i+1]
    a['stop_pair']=0
    a['average_speed']=min1
    l=0
    speedtemp=np.array([])
    nametemp=[]
    stopsnumber=0
    real_stopsnumber=a[a['itcs_stopName']!='-']['itcs_stopName'].count()
    start_time = timeit.default_timer()
    while l<len(a):
        if a['itcs_stopName'][l]!='-':
            nametemp.append(a['itcs_stopName'][l])
            stopsnumber+=1
            if stopsnumber==real_stopsnumber:
                l=len(a)-1
        if nametemp!=[]:
            speedtemp=np.append(speedtemp, a['odometry_vehicleSpeed'][l])
        if(len(nametemp)==2):
            a['stop_pair'][l]=nametemp
            a['average_speed'][l]=np.mean(speedtemp)
            nametemp=[]
            nametemp.append(a['itcs_stopName'][l])
            speedtemp=np.array([])
            speedtemp=np.append(speedtemp, a['odometry_vehicleSpeed'][l])
        l+=1
    elapsed = timeit.default_timer() - start_time
    times.append(elapsed)
    a.drop(a[a['average_speed']==min1].index, inplace=True)
    a.drop(['itcs_stopName', 'odometry_vehicleSpeed'], axis=1, inplace=True)
    
    templist=[a]
    frameslist=frameslist+templist
    f.value += 1
stops_and_average=pd.concat(frameslist)
stops_and_average.reset_index(inplace=True)
print('done!')

print(times)

You can see progress here:


IntProgress(value=0, max=1408)

done!
[0.09734666702570394, 0.5245719159720466, 0.0599565000156872, 0.25264516699826345, 0.5173739170422778, 0.5183220829931088, 0.6492583749932237, 0.6316474170307629, 0.4872289999620989, 0.6194653330021538, 0.2890247500035912, 0.21847554197302088, 0.12310358299873769, 0.28048683295492083, 0.4795418329886161, 0.4429235419956967, 0.5973245410132222, 0.6333108330145478, 0.6723242920124903, 0.5502067089546472, 0.44003362499643117, 0.6406519169686362, 0.5586311249644496, 0.6079845420317724, 0.31410137499915436, 0.25113970797974616, 0.1301163750467822, 0.1358561670058407, 0.22449837502790615, 0.12905762501759455, 0.09928229096112773, 0.5833045829785988, 0.6224836250185035, 0.5779859169851989, 0.4784173750085756, 0.4880985839990899, 0.5861769169569016, 0.29057037498569116, 0.14037024998106062, 0.11839579098159447, 0.46437062497716397, 0.5863822090323083, 0.4626711669843644, 0.5032478750217706, 0.6179770409944467, 0.09350375004578382, 0.6099984170286916, 0.4708265420049429, 0.634316332987509

In [127]:
print(max(times), min(times), np.mean(times), np.median(times))

0.6953254169784486 0.036596582969650626 0.295332859920646 0.26170297898352146


In [102]:
stops_and_average

,index,namefile,stop_pair,average_speed
0,322,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,"[Zürich, Herdernstrasse, Zürich, Hardplatz]",6.640177
1,406,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,"[Zürich, Hardplatz, Zürich, Güterbahnhof]",5.376908
2,508,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,"[Zürich, Güterbahnhof, Zürich, Bäckeranlage]",4.130239
3,564,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,"[Zürich, Bäckeranlage, Zürich, Militär-/Langst...",6.915111
4,619,ZTBus_compressed/B183_2019-04-30_03-18-56_2019...,"[Zürich, Militär-/Langstrasse, Zürich, Kanonen...",4.642477
...,...,...,...,...
443324,12542,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,"[Zürich, Einfangstrasse, Zürich, Zehntenhauspl...",8.122905
443325,12591,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,"[Zürich, Zehntenhausplatz, Zürich, Bahnhof Aff...",3.500344
443326,12636,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,"[Zürich, Bahnhof Affoltern, Zürich, Fronwald]",5.701730
443327,12674,ZTBus_compressed/B208_2022-12-10_23-34-46_2022...,"[Zürich, Fronwald, Zürich, Unteraffoltern]",6.960602


In [86]:
stops_and_average.reset_index(inplace=True)

In [87]:
def useless_func(obj):
    return obj

In [88]:
grouped_stops_and_average=stops_and_average.groupby('namefile').apply(useless_func)

In [89]:
grouped_stops_and_average[['stop_pair', 'average_speed']]

stop_pair  \
namefile                                                                                                       
ZTBus_compressed/B183_2019-04-30_03-18-56_2019-... 0             [Zürich, Herdernstrasse, Zürich, Hardplatz]   
                                                   1               [Zürich, Hardplatz, Zürich, Güterbahnhof]   
                                                   2            [Zürich, Güterbahnhof, Zürich, Bäckeranlage]   
                                                   3       [Zürich, Bäckeranlage, Zürich, Militär-/Langst...   
                                                   4       [Zürich, Militär-/Langstrasse, Zürich, Kanonen...   
...                                                                                                      ...   
ZTBus_compressed/B208_2022-12-10_23-34-46_2022-... 443324  [Zürich, Einfangstrasse, Zürich, Zehntenhauspl...   
                                                   443325  [Zürich, Zehntenhausplatz, Zürich, Bahnhof Aff...   
                                                   443326      [Zürich, Bahnhof Affoltern, Zürich, Fronwald]   
                                                   443327         [Zürich, Fronwald, Zürich, Unteraffoltern]   
                                                   443328    [Zürich, Unteraffoltern, Zürich, Schwandenholz]   

                                                           average_speed  
namefile                                                                  
ZTBus_compressed/B183_2019-04-30_03-18-56_2019-... 0            6.640177  
                                                   1            5.376908  
                                                   2            4.130239  
                                                   3            6.915111  
                                                   4            4.642477  
...                                                                  ...  
ZTBus_compressed/B208_2022-12-10_23-34-46_2022-... 443324       8.122905  
                                                   443325       3.500344  
                                                   443326       5.701730  
                                                   443327       6.960602  
                                                   443328       8.298812  

[443329 rows x 2 columns]